# VAE for text generation

Just a little experiment I'm running to see if I can do it. I have an agentic version of this repo too, but want to make sure I can code it from scratch as well.

My goal is to get a network that can generate synthetic tweets after being trained on an appropriately sized dataset... or at least could, in principle. I'll need to learn more about how to set up recurrent architectures than I currently know, so this is a good exercise for me!

In [1]:
# imports
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from transformers import AutoTokenizer
import kagglehub
from kagglehub import KaggleDatasetAdapter

# The data

First, we get the kaggle dataset, then set up a dataset and dataloader for it.

In [3]:
# Get the kaggle dataset we want.
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "goyaladi/twitter-dataset",
  "twitter_dataset.csv",
)
df.head()

,Tweet_ID,Username,Text,Retweets,Likes,Timestamp
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21


In [ ]:
# Define the dataset class. We need a length and getitem method in addition to the initializer.

# First, define the tokenizer we want to use. We need to do this in advance, since the dataset class
# will do the tokenizing for us.
tokenizer_name = "bert-base-uncased" # Using a recommended tokenizer I saw suggested online.
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

class TweetDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.tweets = df.Text.tolist()
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, idx):
        # We need return both the padded token sets and their original lengths
        # TODO: Double-check this logic.
        tweet = self.tweets[idx]
        tokens = self.tokenizer(
            tweet,
            return_tensors='pt',
        )



# The model

The model itself.

In [ ]:
# TODO: Figure out length logic (tokens or characters, how to compute)
class TextVAE(nn.Module):
    # A VAE that recreates text. Specifically, I'll use it for tweets.

    def __init__(
            self,
            vocab_size,
            embedding_dim,
            hidden_dim,        
            num_layers,
            dropout,
            latent_dim
        ):
        """
        Initialize the VAE.

        Inputs:
        ----
        vocab_size: Size of the model vocabulary. Recommend to set equal 
                    to the number of unique tokens in the training data.
        embedding_dim: Dimension of the space in which tokens are embedded.
        hidden_dim: Dimension of the hidden layer between the GRU and the
                    latent space.
        num_layers: Number of layers in each GRU.
        dropout: Dropout probability for the GRU.
        latent_dim: Dimension of the latent space
        """

        # Initialize the superclass.
        super().__init__()

        # Store latent_dim for later use.
        self.latent_dim = latent_dim

        # Embedding layer: Convert tokens into vectors. Used in both encoder and decoder.
        # ----
        # Input: tensor (batch_size, seq_length)
        # Output: tensor (batch_size, seq_length, embedding_dim)
        self.embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=0,
        )

        # Encoder
        # ----

        # Encoder GRU: Reads tokens into GRU, a recurrent module which can handle the
        # sequence of tokens. Since we're using the hidden states, be mindful to transpose
        # things so that the tensor has the right shape afterwards!
        # Input: tensor (batch_size, seq_length, embedding_dim)
        # Output: output tensor (batch_size, seq_length, hidden_dim)
        #         hidden state tensor (num_layers, batch_size, hidden_dim)
        self.encoder_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=False,
        )

        # Latent space
        # ----

        # Fully connected layers: One each for mu and log variance. Note that 
        # this only uses the hidden state of the last token in the sequence! 
        # The relevant dimensional shuffling is done in the encoding function. 
        # In both cases:
        # Input: tensor (batch_size, hidden_dim)
        # Output: tensor (batch_size, latent_dim)
        self.fc_mu = nn.Linear(
            in_features=hidden_dim, 
            out_features=latent_dim,
        )
        self.fc_logvar = nn.Linear(
            in_features=hidden_dim, 
            out_features=latent_dim,
        )

        # Decoder
        # ----
        
        # Fully connected layer: Going from the latent space to the decoder's GRU.
        # Input: tensor (batch_size, seq_length, latent_dim)
        # Output: tensor (batch_size, seq_length, hidden_dim)
        self.fc_decoder_init = nn.Linear(
            in_features=latent_dim,
            out_features=hidden_dim
        )

        # # Decoder embedding layer: Convert max hidden state indices (i.e. tokens) into vectors
        # # Input: tensor (batch_size, seq_length)
        # # Output: tensor (batch_size, seq_length, embedding_dim)
        # self.decoder_embedding = nn.Embedding(
        #     vocab_size,
        #     embedding_dim,
        #     padding_idx=0,
        # )

        # Decoder GRU: The recurrent part of the decoder. Just like the encoder!
        # Input: tensor (batch_size, seq_length, embedding_dim)
        # Output: output tensor (batch_size, seq_length, hidden_dim)
        #         hidden state tensor (num_layers, batch_size, hidden_dim)
        self.decoder_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=False,
        )

        # Output layer: Convert GRU hidden states into token logits. Note that 
        # this only uses the hidden state of the last token in the sequence! 
        # The relevant dimensional shuffling is done in the decoding function.
        # Input: tensor (batch_size, hidden_dim)
        # Output: tensor (batch_size, vocab_size)
        self.fc_decoder_output = nn.Linear(
            in_features=hidden_dim,
            out_features=vocab_size
        )
    
    def encode(
            self, 
            x, 
            lengths
        ):
        """
        Encode input sequences into a latent distribution, i.e. mu and log variance.

        Inputs:
        ----
        x: Input tensor of token indices. Shape: (batch_size, seq_length)
        lengths: Actual lengths of the sequences before padding. Shape: (batch_size)

        Outputs:
        ----
        mu, logvar: Tensors representing the parameters of the latent distribution. Shape: (batch_size, latent_dim) 
        """

        # Embed the sequences.
        embedded = self.embedding(x)


        # Pack the sequences for GRU processing. This is a PyTorch data structure that
        # helps the GRU ignore padding tokens and process things faster.
        packed = nn.utils.rnn.pack_padded_sequence(
            input=embedded,
            lengths=lengths.cpu(),
            batch_first=True,
            enforce_sorted=False,
        )

        # Encode the packed sequence through the GRU.
        _, hidden = self.encoder_gru(packed)

        # Get the hidden state of the last layer.
        last_hidden = hidden[-1:].squeeze(0).contiguous()
        last_hidden = last_hidden.view(last_hidden.size(0), -1)

        # Get the mean and variance.
        mu = self.fc_mu(last_hidden)
        logvar = self.fc_logvar(last_hidden)

        return mu, logvar

    def latent_sample(
            self, 
            mu, 
            logvar
        ):
        """
        Sample from the latent distribution using the reparameterization trick.

        Inputs:
        ----
        mu: Mean tensor of the latent distribution. Shape: (batch_size, latent_dim)
        logvar: Log variance tensor of the latent distribution. Shape: (batch_size, latent_dim)

        Outputs:
        ----
        z: Sampled latent tensor. Shape: (batch_size, latent_dim)
        """

        # Get the standard deviation.
        std = torch.exp(0.5 * logvar)

        # Add Gaussian noise.
        eps = torch.randn_like(std)

        # Now return the latent vector.
        z = mu + eps * std

        return z
    
    def decode(
            self, 
            z, 
            max_length
):
        """
        Decode latent vectors into output token logits.
        
        Inputs:
        ----
        z: Latent tensor. Shape: (batch_size, latent_dim)
        max_length: Maximum length of the output sequences to generate.
        
        Outputs:
        ----
        logits: Output token logits: Shape: (batch_size, max_length, vocab_size)
        """

        # Get the batch size from the latent tensor.
        batch_size = z.size(0)

        # Get the initial hidden state for the decoder GRU.
        decoder_hidden = self.fc_decoder_init(z)
        decoder_hidden = decoder_hidden.view(
            batch_size, self.hidden_dim
        ).contiguous()

        # Prepare the input tokens for the decoder GRU. Start with all start of
        # sequence tokens.
        current_input = torch.full(
            size=(batch_size, 1), 
            fill_value=1, 
            dtype=torch.long,
            device=z.device
        )

        # Prepare a list to collect the logits.
        logits = []

        # Produce a logit for each possible max token.
        for _ in range(max_length):

            # Embed the current inputs.
            embedded = self.embedding(current_input)

            # Decode the embedding through the GRU. The old value
            # of decoder_hidden is used to produce the new one.
            output, decoder_hidden = self.decoder_gru(embedded, decoder_hidden)

            # Send the output through to get the logits. Squeeze the 1st
            # dimension (the seq_length), since we're processing one token
            # at a time, and need to match the expected input shape of the 
            # fully connected layer.
            out = self.fc_decoder_output(output.squeeze(1))
            logits.append(out)

        # Finally, stack the logits and return them. Stack vs. cat here, since
        # we're adding the sequence length back in as a new dimension.
        logits = torch.stack(logits, dim=1)
        return logits
        
    def forward(self, x, lengths, max_length=None):
        """
        The network's forward pass. Returns both the reconstructed logits and
        the mu and logvar parameters of the latent distribution. We need both
        to calculate the loss.

        Inputs:
        ----
        x: Input sequences. Shape: (batch_size, seq_length)
        lengths: Actual lengths of each sequence.
        max_lengths: Maximum length for decoder, defaults to input length

        Outputs:
        ----
        logits: Logits of the reconstruted sequences. 
        Shape: (batch_size, max_length, vocab_size)
        mu: Mean vectors of the latent distributions. Shape: (batch_size, 
        latent_dim).
        logvar: Log variance vectors of the latent distributions. Shape: 
        (batch_size, latent_dim).
        """
        
        # Set the default max_length.
        if max_length is None:
            max_length = x.size(1)
        
        # Encode the input sequences.
        mu, logvar = self.encode(x, lengths)

        # Sample from the latent distribution.
        z = self.latent_sample(mu, logvar)

        # Now decode the latent vectors.
        logits = self.decode(z, max_length)

        return logits, mu, logvar
    
    def generate(self, num_samples, max_length, mu = None, logvar = None):
        """
        Generate samples from the latent space.

        Inputs:
        ----
        num_samples: Number of samples to generate.
        max_length: Maximum length of each sample, in tokens.
        mu: optional mean tensor for the latent distribution. If None, 
            samples will be drawn from a standard normal distribution.
        logvar: optional log variance tensor for the latent distribution.
                If None, samples will be drawn from a standard normal
                distribution.

        Outputs:
        ----
        tokens: Generated samples' token indices. Shape: (num_samples, max_length)
        """
        # Set default mu and logvar values, if necessary.
        if mu is None:
            mu = torch.zeros((1, self.latent_dim), device=next(self.parameters()).device)
        if logvar is None:
            logvar = torch.zeros((1, self.latent_dim), device=next(self.parameters()).device)

        # Expand mu and logvar to match num_samples. Shape will
        # be (num_samples, latent_dim)
        mu = mu.expand(num_samples, -1)
        logvar = logvar.expand(num_samples, -1)

        # Set into eval mode, and disable gradient calculations.
        self.eval()
        with torch.no_grad():
            # Sample from the distribution.
            z = self.latent_sample(mu, logvar)

            # Now decode the latent vectors and get the logits. Shape
            # is 
            logits = self.decode(z, max_length)

            # Get the token indices by grabbing the largest logits' 
            # indices. Do across the last dimension, i.e. the vocab
            # dimension.
            tokens = torch.argmax(logits, dim=-1)

        return tokens

Now the loss function, which incorporates both reconstruction loss and KL-divergence from the prior distribution we want. The loss function is really the evidence lower bound (ELBO) of variational inference.

In [11]:
def vae_loss(
    recon_logits,
    target_tokens,
    mu,
    logvar,
    prior_mu=None,
    prior_logvar=None
):
    """
    Calculate the VAE's loss, combining reconstruction loss and KL
    divergence from a prior distribution.

    Inputs:
    ----
    recon_logits: Reconstructed logits from the VAE. Shape: (batch_size, seq_length, vocab_size)
    target_tokens: Target token indices, the ones we want the logits to be
                   close to. Shape: (batch_size, seq_length)
    mu: Mean vectors of the latent distributions. Shape: (batch_size, latent_dim)
    logvar: Log variance vectors of the latent distributions. Shape: (batch_size, latent_dim)
    prior_mu: Mean of the prior distribution, default unit normal. Shape: (latent_dim)
    prior_logvar: Log variance of the prior distribution, default unit normal.
                  Shape: (latent_dim)
    
    Outputs:
    ----
    loss: The VAE loss.
    recon_loss: The reconstruction part of the loss.
    kl_div: The KL divergence part of the loss.
    """

    # First, get the shape of the reconstructed logits so we can get
    # the batch size, sequence length, and vocabulary size.
    batch_size, seq_length, vocab_size = recon_logits.shape
    latent_dim = mu.shape[1]

    # Now set up the prior distribution parameters, if needed.
    # TODO: Add logvar
    if prior_mu is None:
        prior_mu = torch.zeros((1, latent_dim), device=recon_logits.device)
    if prior_logvar is None:
        prior_logvar = torch.zeros((1, latent_dim), device=recon_logits.device)


    # Now we flatten the logits and target tokens. Remember, at this
    # point, we don't care about keeping the sequences separate; we
    # just need to know how close each logit is to its target token.
    logits_flat = recon_logits.view(-1, vocab_size) # One value per possible token, per token.
    targets_flat = target_tokens.view(-1) # Just a flat list of target tokens.

    # Calculate the reconstruction loss as cross-entropy.
    recon_loss = cross_entropy(logits_flat, targets_flat)

    # Now calculate the KL divergence. This uses a closed-form solution
    # for the KL divergence between two Gaussians.
    kl_div = 0.5 * torch.sum(
        torch.exp(logvar - prior_logvar) + (prior_mu - mu) ** 2
        / torch.exp(prior_logvar) - logvar + prior_logvar - 1
    )

    # Finally, return the completed loss.
    loss = recon_loss + kl_div

    return loss, recon_loss, kl_div

Now define the training loop we will use.

In [ ]:
def train_vae(
    model,
    train_loader,
    optimizer,
    epochs=1,
):
    """
    Train a VAE on a set of training data.

    Inputs:
    ----
    model: The VAE to be trained.
    train_loader: The dataloader for our training dataset.
    optimizer: The optimizer we're using.
    epochs: Number of epochs to train for.

    Outputs:
    ----
    None
    """

    # Set the model to training mode.
    model.train()

    # Iterate over epochs.
    for epoch in range(epochs):

        # Iterate over batches in the data loader.
        for batch in train_loader:
            # Get the tweets and their lengths.
            # TODO: Figure out lengths computation; perhaps earlier?
            tweets = batch
            lengths = 

# The data

Now, we load the data.

In [4]:
# Load the dataset.
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "goyaladi/twitter-dataset",
    "twitter_dataset.csv"
)
df.head()


,Tweet_ID,Username,Text,Retweets,Likes,Timestamp
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21


In [ ]:
# Define the torch dataset.
class TwitterDataset(Dataset):
    """
    Defining a torch dataset for our tweets.
    """
    def __init__(
        self,
        df
    ):
        # Just store the tweets.
        self.tweets = df["Text"].to_list()

    def __len__(self):
        # Define the length of the dataset.
        return len(self.tweets)
    
    def __getitem__(self, idx):
        # Return the tweet at the specified index.
        return self.tweets[idx]

In [8]:
# Now create the dataloader
dataset = TwitterDataset(df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)